# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('./Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [4]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df.loan_status

In [5]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [7]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [15]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,y
    , random_state=1
    , stratify=y
)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [16]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler_std = StandardScaler()

In [25]:
def Combine_Names_for_CF_Matrix(names=None, y_pred=None):
    if names is not None:
        lst = names
    elif y_pred is not None:
        lst = sorted(set(y_pred))

    idx = [f"Actual {x}" for x in lst]
    col = [f"Prediction {x}" for x in lst]

    return idx,col

def Generate_Confusion_Matrix(y_test, y_pred, names=None):
    from sklearn.metrics import confusion_matrix

    idx,col = Combine_Names_for_CF_Matrix(names=names, y_pred=y_pred)

    cm_df = pd.DataFrame(
        data=confusion_matrix(y_test, y_pred)
        , index= idx
        , columns= col
    )
    cm_df['Total Predictions'] = cm_df.sum(axis='columns')
    cm_df.loc["Total Actuals"] = cm_df.sum(axis='index')

    return cm_df
def Calc_Conf_Matrix(cm_df:pd.DataFrame):
    true_negative  = cm_df.iloc[0,0]
    false_positive = cm_df.iloc[0,1]
    false_negative = cm_df.iloc[1,0]
    true_positive  = cm_df.iloc[1,1]
    total_predictions  = cm_df.loc['Total Actuals','Total Predictions']
    total_predict_yes  = cm_df.iloc[2,1]
    total_predict_no     = cm_df.iloc[2,0]
    
    total_actuals_positive = cm_df.iloc[1,2]
    total_actuals_negaitive = cm_df.iloc[0,2]

    accuracy                =  (true_positive  + true_negative ) /total_predictions
    misclassification_rate  =  (false_positive + false_negative) /total_predictions  # Error Rate
    sensitivity_rate        =  true_positive / total_actuals_positive                # True Positive Rate or Recall
    false_positive_rate     =  false_positive / total_actuals_negaitive
    true_negative_rate      =  true_negative  / total_actuals_negaitive              # Specificity
    precision               =  true_positive /  total_actuals_positive

    return {'accuracy':accuracy
            ,'misclassification_rate':misclassification_rate
            ,'sensitivity_rate':sensitivity_rate
            ,'false_positive_rate':false_positive_rate
            ,'true_negative_rate':true_negative_rate
            ,'precision':precision}

def Describe_Confusion_Matrix(cm_df:pd.DataFrame):
    results = Calc_Conf_Matrix(cm_df=cm_df)
    display(cm_df)

    print(f"Overall, how often is the classifier correct (Accuracy): {results['accuracy']:.3%}")
    print(f"Overall, how often is it wrong (Misclassification Rate or Error Rate): {results['misclassification_rate']:.3%}")
    print(f"When it's actually yes, how often does it predict yes? (True Positive Rate or Sensitivity or Recall): {results['sensitivity_rate']:.3%}")
    print(f"When it's actually no, how often does it predict no (False Posive Rate): {results['true_negative_rate']:.3%}")
    print(f"When it predicts yes, how often is it correct? (Precision):{results['precision']:.3%}")
    print(f"When it's actually no, how often does it predict no? (False Positive Rate): {results['false_positive_rate']:.3%}")
    

In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

def Categorize_Txt_Cols(df,col_name):
    """Categorizes the columns with numbers"""
    le.fit(df[col_name])
    return le.transform(df[col_name])

def Get_Columns_With_Text(df):
    """Get's alist of text columns from a DataFrame"""
    list_of_columns = list(zip(list(df.keys()), df.dtypes.to_list()))
    return [col[0] for col in list_of_columns if col[1]==np.dtype('O')]

def Convert_Text_Cols(df):
    """Determines which columns are Text"""
    list_of_text_columns = Get_Columns_With_Text(df)
    
    for col in list_of_text_columns:
        df[col] = Categorize_Txt_Cols(df,col_name=col)
    
    return df

X_train = Convert_Text_Cols(X_train)
X_test  = Convert_Text_Cols(X_test)

In [19]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler_std = scaler_std.fit(X_train)

In [20]:
# Scale the training and testing data
X_train_scaled_std = X_scaler_std.transform(X_train)
X_test_scaled_std  = X_scaler_std.transform(X_test)

# Simple Logistic Regression

In [21]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [23]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
print(f"The Balanced Accuracy Score for Simple Logic Regression: {balanced_accuracy_score(y_test, y_pred):.4%}")

The Balanced Accuracy Score for Simple Logic Regression: 95.4321%


In [26]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
#confusion_matrix(y_test, y_pred)
cm_df = Generate_Confusion_Matrix(y_test, y_pred)
Describe_Confusion_Matrix(cm_df)

,Prediction high_risk,Prediction low_risk,Total Predictions
Actual high_risk,571,54,625
Actual low_risk,93,18666,18759
Total Actuals,664,18720,19384


Overall, how often is the classifier correct (Accuracy): 99.242%
Overall, how often is it wrong (Misclassification Rate or Error Rate): 0.758%
When it's actually yes, how often does it predict yes? (True Positive Rate or Sensitivity or Recall): 99.504%
When it's actually no, how often does it predict no (False Posive Rate): 91.360%
When it predicts yes, how often is it correct? (Precision):99.504%
When it's actually no, how often does it predict no? (False Positive Rate): 8.640%


In [69]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred,digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8599    0.9136    0.9950    0.8860    0.9535    0.9017       625
   low_risk     0.9971    0.9950    0.9136    0.9961    0.9535    0.9165     18759

avg / total     0.9927    0.9924    0.9162    0.9925    0.9535    0.9160     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [32]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_samp, y_samp = ros.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_train), Counter(y_samp)

(Counter({'low_risk': 56277, 'high_risk': 1875}),
 Counter({'low_risk': 56277, 'high_risk': 56277}))

In [36]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

mdl = LogisticRegression(solver='lbfgs', random_state=1)
mdl.fit(X_samp, y_samp)
y_pred_samp = mdl.predict(X_test)

In [37]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

print(f"Balanced accuracy score is: {balanced_accuracy_score(y_test, y_pred_samp):.4%}")

Balanced accuracy score is: 99.4801%


In [38]:
# Display the confusion matrix
cm_df_samp = Generate_Confusion_Matrix(y_test, y_pred_samp)
Describe_Confusion_Matrix(cm_df_samp)

,Prediction high_risk,Prediction low_risk,Total Predictions
Actual high_risk,622,3,625
Actual low_risk,105,18654,18759
Total Actuals,727,18657,19384


Overall, how often is the classifier correct (Accuracy): 99.443%
Overall, how often is it wrong (Misclassification Rate or Error Rate): 0.557%
When it's actually yes, how often does it predict yes? (True Positive Rate or Sensitivity or Recall): 99.440%
When it's actually no, how often does it predict no (False Posive Rate): 99.520%
When it predicts yes, how often is it correct? (Precision):99.440%
When it's actually no, how often does it predict no? (False Positive Rate): 0.480%


In [68]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_samp,digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8556    0.9952    0.9944    0.9201    0.9948    0.9897       625
   low_risk     0.9998    0.9944    0.9952    0.9971    0.9948    0.9896     18759

avg / total     0.9952    0.9944    0.9952    0.9946    0.9948    0.9896     19384



### SMOTE Oversampling

In [40]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_smote, y_smote = SMOTE(random_state=1,sampling_strategy=1.0).fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_smote)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [44]:
# Train the Logistic Regression model using the resampled data
mdl_smote = LogisticRegression(solver='lbfgs', random_state=1)
mdl_smote.fit(X_smote, y_smote)

LogisticRegression(random_state=1)

In [46]:
# Calculated the balanced accuracy score
y_pred_smote = mdl_smote.predict(X_test)
print(f"The balance accuracy score for SMOTE is: {balanced_accuracy_score(y_test, y_pred_smote):.4%}")

The balance accuracy score for SMOTE is: 99.4828%


In [47]:
# Display the confusion matrix
cm_df_smote = Generate_Confusion_Matrix(y_test, y_pred_smote)
Describe_Confusion_Matrix(cm_df_smote)

,Prediction high_risk,Prediction low_risk,Total Predictions
Actual high_risk,622,3,625
Actual low_risk,104,18655,18759
Total Actuals,726,18658,19384


Overall, how often is the classifier correct (Accuracy): 99.448%
Overall, how often is it wrong (Misclassification Rate or Error Rate): 0.552%
When it's actually yes, how often does it predict yes? (True Positive Rate or Sensitivity or Recall): 99.446%
When it's actually no, how often does it predict no (False Posive Rate): 99.520%
When it predicts yes, how often is it correct? (Precision):99.446%
When it's actually no, how often does it predict no? (False Positive Rate): 0.480%


In [71]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smote,digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8567    0.9952    0.9945    0.9208    0.9948    0.9898       625
   low_risk     0.9998    0.9945    0.9952    0.9971    0.9948    0.9896     18759

avg / total     0.9952    0.9945    0.9952    0.9947    0.9948    0.9896     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [49]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_cc, y_cc = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_cc)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [53]:
# Train the Logistic Regression model using the resampled data
mdl_cc = LogisticRegression(solver='lbfgs', random_state=1)
mdl_cc.fit(X_cc, y_cc)

LogisticRegression(random_state=1)

In [54]:
# Calculate the balanced accuracy score
y_pred_cc = mdl_cc.predict(X_test)
print(f"The balance accuracy score for Cluster Centroids is: {balanced_accuracy_score(y_test, y_pred_cc):.4%}")

The balance accuracy score for Cluster Centroids is: 98.3681%


In [55]:
# Display the confusion matrix
cm_df_cc = Generate_Confusion_Matrix(y_test, y_pred_cc)
Describe_Confusion_Matrix(cm_df_cc)


,Prediction high_risk,Prediction low_risk,Total Predictions
Actual high_risk,608,17,625
Actual low_risk,102,18657,18759
Total Actuals,710,18674,19384


Overall, how often is the classifier correct (Accuracy): 99.386%
Overall, how often is it wrong (Misclassification Rate or Error Rate): 0.614%
When it's actually yes, how often does it predict yes? (True Positive Rate or Sensitivity or Recall): 99.456%
When it's actually no, how often does it predict no (False Posive Rate): 97.280%
When it predicts yes, how often is it correct? (Precision):99.456%
When it's actually no, how often does it predict no? (False Positive Rate): 2.720%


In [72]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_cc,digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8563    0.9728    0.9946    0.9109    0.9836    0.9654       625
   low_risk     0.9991    0.9946    0.9728    0.9968    0.9836    0.9696     18759

avg / total     0.9945    0.9939    0.9735    0.9940    0.9836    0.9695     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [59]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
smot_en = SMOTEENN(random_state=1)
X_smot_en, y_smot_en = smot_en.fit_resample(X_train,y_train)
# View the count of target classes with Counter
Counter(y_smot_en)

Counter({'high_risk': 55509, 'low_risk': 55930})

In [64]:
# Train the Logistic Regression model using the resampled data
mdl_smot_en = LogisticRegression(solver='lbfgs', random_state=1)
mdl_smot_en.fit(X_smot_en, y_smot_en)

LogisticRegression(random_state=1)

In [65]:
# Calculate the balanced accuracy score
y_pred_smot_enn = mdl_smot_en.predict(X_test)
print(f"The balanced accuracy score for the SMOTEENN is: {balanced_accuracy_score(y_test, y_pred_smot_enn):.4%}")

The balanced accuracy score for the SMOTEENN is: 99.4748%


In [66]:
# Display the confusion matrix
cm_df_smot_enn = Generate_Confusion_Matrix(y_test, y_pred_smot_enn)
Describe_Confusion_Matrix(cm_df_smot_enn)

,Prediction high_risk,Prediction low_risk,Total Predictions
Actual high_risk,622,3,625
Actual low_risk,107,18652,18759
Total Actuals,729,18655,19384


Overall, how often is the classifier correct (Accuracy): 99.433%
Overall, how often is it wrong (Misclassification Rate or Error Rate): 0.567%
When it's actually yes, how often does it predict yes? (True Positive Rate or Sensitivity or Recall): 99.430%
When it's actually no, how often does it predict no (False Posive Rate): 99.520%
When it predicts yes, how often is it correct? (Precision):99.430%
When it's actually no, how often does it predict no? (False Positive Rate): 0.480%


In [73]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smot_enn,digits=4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8532    0.9952    0.9943    0.9188    0.9947    0.9896       625
   low_risk     0.9998    0.9943    0.9952    0.9971    0.9947    0.9894     18759

avg / total     0.9951    0.9943    0.9952    0.9945    0.9947    0.9894     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   The SMOTE has the highest Balance Accuracy Score with 99.48%

2. Which model had the best recall score?

    Simple Logic Regression

3. Which model had the best geometric mean score?

    Combination Over and Under Sampling
